In [32]:
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, Compose
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.datasets.UAVidSemanticSegmentationDataset import (
    UAVidSemanticSegmentationDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

## Prepare environment

In [33]:
torch.cuda.is_available()

True

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [35]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42
UAVID_DATASET_PATH = "data/UAVidSemanticSegmentationDataset"
IMAGE_SIZE = 576

In [36]:
train_dataset = UAVidSemanticSegmentationDataset(
    UAVID_DATASET_PATH,
    transforms=[Compose([Resize(IMAGE_SIZE), ResizeToDivisibleBy32()])],
)
print(len(train_dataset))

200


In [37]:
val_dataset = UAVidSemanticSegmentationDataset(
    UAVID_DATASET_PATH,
    split="valid",
    transforms=[Compose([Resize(IMAGE_SIZE), ResizeToDivisibleBy32()])],
)
print(len(val_dataset))

70


In [38]:
test_dataset = UAVidSemanticSegmentationDataset(
    UAVID_DATASET_PATH,
    split="test",
    transforms=[Compose([Resize(IMAGE_SIZE), ResizeToDivisibleBy32()])],
)
print(len(test_dataset))

10


## Sanity check data

In [39]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [40]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([1, 3, 576, 1024])
torch.Size([1, 3, 576, 1024])


In [41]:
for images, masks in val_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([1, 3, 576, 1024])
torch.Size([1, 3, 576, 1024])


In [42]:
for images in test_loader:
    print(images.shape)
    break

torch.Size([1, 3, 576, 1024])


In [ ]:
to_pil_transform = transforms.ToPILImage()

In [ ]:
img = to_pil_transform(images.squeeze())

In [ ]:
# img.show()

In [ ]:
msk = to_pil_transform(masks.squeeze())

In [ ]:
# msk.show()

## Run example model on single image

In [ ]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=8,  # model output channels (number of classes in your dataset)
).to(device)

In [ ]:
baseline_model = BaselineModel(classes=8).to(device)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    # output = (output > 0.5).float()

    output_baseline = baseline_model(images.to(device))
    # output_baseline = (output_baseline > 0.5).float()

In [ ]:
print(output.shape)

In [ ]:
print(output_baseline.shape)

In [ ]:
unique, counts = np.unique(output.cpu(), return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
evaluate_result(output, masks)

In [ ]:
evaluate_result(output_baseline, masks)

## Run example model on dataset

In [ ]:
model.eval()
metrics_dict = {
    "iou": [],
    "f1": [],
    "accuracy": [],
    "recall": [],
}

In [ ]:
with torch.no_grad():
    for images, masks in train_loader:
        output = model(images.to(device))
        # output = (output > 0.5).float()

        iter_metrics = evaluate_result(output, masks)
        for key in metrics_dict.keys():
            metrics_dict[key].append(iter_metrics[key])

In [ ]:
print(metrics_dict)